### Import libraries and functions

In [117]:
from vis_utils import load_volume, VolumeVisualizer, ColorMapVisualizer, draw_graph_on_model
from identification import get_data_graph, generate_sphere, clean_data_graph, parametrize_graph, get_vessel_graph, remove_graph_components, connect_endings_3d, get_starting_points,sphere_full, get_points_of_interest

In [118]:
import pickle
import numpy as np
# import matplotlib.pyplot as plt
from skimage import measure
from scipy.ndimage import zoom
from skimage.morphology import ball
# from scipy.signal import fftconvolve
from scipy.ndimage import distance_transform_edt
# from skimage.morphology import skeletonize_3d, binary_dilation, convex_hull_image, ball, area_closing

### Flags

In [18]:
visualize_steps = True 

### Utils

In [119]:
def visualize_mask_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8), binary=True).visualize()
    
def visualize_mask_non_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8) * 255, binary=False).visualize()

### Read data

In [120]:
volume = np.fromfile('../data/P13/data.raw', dtype=np.uint8)
volume = volume.reshape(877, 488, 1132)

### Cut image

In [121]:
volume = volume[200:700, :, 100:650]
val = 20

volume[:, 200:, -35:] = val
volume[-150:, 200:, -100:-35] = val
volume[:, -200:-150, -100:-35] = val
volume[:, :210, -50:] = val
volume[:150, :150, -85:-50] = val

volume = volume[-170:-40,200:480,30:260]
volume[-90:, :, -110:] = val
volume[:, -80:, -92:] = val

In [122]:
volume.shape

(130, 280, 230)

### Segmentation

In [123]:
mask = volume > 31

### Vizualization

In [54]:
if visualize_steps:
    VolumeVisualizer(mask, binary=True).visualize() 

In [124]:
scaled_mask = zoom(mask, zoom=0.7, order=0)

In [125]:
scaled_mask.shape

(91, 196, 161)

### EDT transform

In [57]:
%%time
edt_img = distance_transform_edt(scaled_mask)

CPU times: total: 484 ms
Wall time: 496 ms


In [126]:
if visualize_steps:
    ColorMapVisualizer(edt_img.astype(np.uint8)).visualize()

### Graph creation

In [127]:
import numpy as np
import networkx as nx

from scipy.ndimage import distance_transform_edt
from scipy.spatial.distance import euclidean
from skimage.draw import circle_perimeter, disk
from skimage.feature import peak_local_max

def get_points_of_interest(point, radius, visited, edt_img):
    """Get list of points that can be on the vessel centerline. The method utilizes local maxima
    to find the centerline on the basis of the EDT image of the vessel. The returned points are
    sorted by the EDT image value, with the pixels on the largest vessel at the beginning of the
    list.

    Args:
        point (tuple(int, int)): Point on centerline.
        radius (int): Radius of search circle.
        visited (np.array(bool)): Boolean array of visited image pixels.

    Returns:
        list: A list of points that can be on the vessel centerline.
    """
    # get points on circle circumference
    #circle_points = circle_perimeter(*point, radius=radius, method="andres")
    circle_points = get_points_on_sphere(point, radius)
    circle_points = list(filter(lambda x: is_inside_image(x, edt_img.shape), circle_points))

    #print("Circle points: ", len(circle_points), edt_img.shape)
    # create image with edt values of point on circle circumference
    helper = np.zeros_like(edt_img)
    
    for p in circle_points:
        helper[p] = edt_img[p]
    # find local maxima
    coords = peak_local_max(
        helper, min_distance=radius // 3, threshold_abs=2, threshold_rel=0.5
    )
    #print("Local peaks on surface: ", len(coords))
    # find coordinates of local 
    
    for p in circle_points:
        helper[p] = 0
    helper[tuple(coords.T)] = 1
    helper[visited] = 0
    points_of_interest = list(zip(*np.nonzero(helper)))
    
    points_of_interest = list(filter(lambda x: is_inside_image(x, edt_img.shape), points_of_interest))
    # sort points by vessel diameter
    points_of_interest.sort(key=lambda x: edt_img[x], reverse=True)
    return points_of_interest


def get_points_on_sphere(center, r):
    b = generate_sphere(r)
    non_zero = np.array(tuple(zip(*np.nonzero(b))))
    new_points = []

    for point in non_zero:
        x,y,z = point[0], point[1], point[2]
        xc, yc, zc = center

        x,y,z = x-r, y-r, z-r
        x,y,z = x + xc, y + yc, z + zc
        new_points.append((x,y,z))
    return new_points


def is_inside_image(p, shape):
    return p[0] >= 0 and p[1] >= 0 and p[2] >= 0 and p[0] < shape[0] and p[1] < shape[1] and p[2] < shape[2]

def generate_sphere(radius):
    large = ball(radius)
    small = ball(np.maximum(1, radius-1))
    padded_small = np.pad(small, 1, mode='constant')
    sphere = large - padded_small
    return sphere

def get_vessel_graph(seg_img):
    """Produce a graph which describes the vessel structure.

    Args:
        seg_img (np.array): Segmentation of vessel image.

    Returns:
        nx.Graph: Directed graph describing the vessel structure.
    """
    # perform euclidean distance transform on base image
    edt_img = distance_transform_edt(seg_img)
    # get starting points
    starting_point_stack = get_starting_points(edt_img)
    # create vessel tree data structure
    vessel_graph = nx.DiGraph()
    # create array for holding information on visited pixels
    visited = np.zeros_like(edt_img).astype(bool)
    
    i = 0
    print("Starting points: ", len(starting_point_stack))
    while starting_point_stack:
        # select new starting point and check if it was not visited already
        starting_point = starting_point_stack.pop()
        if visited[starting_point]:
            continue

        vessel_graph.add_node(starting_point)
        points_to_examine = [starting_point]
        
        j = 0
        while points_to_examine:
            if j % 10 == 0:
                print(f"i: {i} j: {j} {len(points_to_examine)}")
            # get point from queue
            point = points_to_examine.pop()
            # calculate vessel radius
            radius = int(edt_img[point])
            # if radius is too small, don't go further into this vessel
            # also limited by min_distance having to be >= 1 in peak_local_max
            if radius <= 2:
                continue
            # get points that can be on centreline
            points_of_interest = get_points_of_interest(point, 3*radius, visited, edt_img)

            # eliminate points that are too close to each other
            for poi_1 in points_of_interest:
                for poi_2 in points_of_interest:
                    if poi_1 != poi_2 and euclidean(poi_1, poi_2) < edt_img[poi_1]:
                        points_of_interest.pop(points_of_interest.index(poi_2))
            # add points of interest to examination list and to vessel graph
            for poi in points_of_interest:
                # avoid node duplication
                if not (poi in points_to_examine or vessel_graph.has_node(poi)):
                    points_to_examine.append(poi)
                    vessel_graph.add_edge(point, poi)
            # remove potential centreline pixels next to analyzed point to prevent going backwards
            disk_points = sphere_full(point, 2 * radius, visited.shape)
            visited[disk_points] = True
            j += 1
        i += 1

    return vessel_graph.to_undirected()

In [128]:
%%time
vessel_graph = get_vessel_graph(scaled_mask)

Starting points:  4286
i: 0 j: 0 1
i: 0 j: 10 4
i: 0 j: 20 9
i: 0 j: 20 8
i: 0 j: 20 7
i: 0 j: 20 6
i: 0 j: 30 8
i: 0 j: 30 7
i: 0 j: 30 6
i: 0 j: 40 1
i: 1 j: 0 1
i: 2 j: 0 1
i: 3 j: 0 1
i: 4 j: 0 1
i: 5 j: 0 1
i: 6 j: 0 1
i: 7 j: 0 1
i: 8 j: 0 1
i: 9 j: 0 1
i: 10 j: 0 1
i: 11 j: 0 1
i: 12 j: 0 1
i: 13 j: 0 1
i: 14 j: 0 1
i: 15 j: 0 1
i: 16 j: 0 1
i: 17 j: 0 1
i: 18 j: 0 1
i: 19 j: 0 1
i: 20 j: 0 1
i: 21 j: 0 1
i: 22 j: 0 1
i: 23 j: 0 1
i: 24 j: 0 1
i: 25 j: 0 1
i: 26 j: 0 1
i: 27 j: 0 1
i: 28 j: 0 1
i: 29 j: 0 1
i: 30 j: 0 1
i: 31 j: 0 1
i: 32 j: 0 1
i: 33 j: 0 1
i: 34 j: 0 1
i: 35 j: 0 1
i: 36 j: 0 1
i: 37 j: 0 1
i: 38 j: 0 1
i: 39 j: 0 1
i: 40 j: 0 1
i: 41 j: 0 1
i: 42 j: 0 1
i: 43 j: 0 1
i: 44 j: 0 1
i: 45 j: 0 1
i: 46 j: 0 1
i: 47 j: 0 1
i: 48 j: 0 1
i: 49 j: 0 1
i: 50 j: 0 1
i: 51 j: 0 1
i: 52 j: 0 1
i: 53 j: 0 1
i: 54 j: 0 1
i: 55 j: 0 1
i: 56 j: 0 1
i: 57 j: 0 1
i: 58 j: 0 1
i: 59 j: 0 1
i: 60 j: 0 1
i: 61 j: 0 1
i: 62 j: 0 1
i: 63 j: 0 1
i: 64 j: 0 1
i: 65 j: 0 1
i: 66 j: 0 1

In [129]:
print('Number of nodes', len(vessel_graph.nodes))
print('Number of edges', len(vessel_graph.edges))
print('Average degree', sum(dict(vessel_graph.degree).values()) / len(vessel_graph.nodes))

Number of nodes 3577
Number of edges 134
Average degree 0.07492311993290467


In [107]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(scaled_mask, vessel_graph)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

### Remove components

In [130]:
%%time
vessel_graph_rm = remove_graph_components(vessel_graph)

CPU times: total: 15.6 ms
Wall time: 11 ms


In [131]:
# print(nx.info(vessel_graph_rm))

print('Number of nodes', len(vessel_graph_rm.nodes))
print('Number of edges', len(vessel_graph_rm.edges))
print('Average degree', sum(dict(vessel_graph_rm.degree).values()) / len(vessel_graph_rm.nodes))

Number of nodes 145
Number of edges 134
Average degree 1.8482758620689654


In [66]:
if visualize_steps:
    model_with_graph = draw_graph_on_model(scaled_mask, vessel_graph_rm)
    ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

### Connect endings 3D

In [134]:
from queue import SimpleQueue

# Orignal Witold method adjusted to 3D

def connect_endings_3d(graph, edt_img, multiplier=2.5):
    """Fix discontinuities in vessel graph.

    Args:
        graph (nx.Graph): Graph object from NetworkX library.
        edt_img (np.array): Numpy array containing euclidean distance transformed image of vessel.
        multiplier (int, optional): Multiplier for maximum distance between nodes to connect.
        Defaults to 2.

    Returns:
        nx.Graph: Graph without discontinuities.
    """
    # find potential discontinuities
    endings = [node for node, degree in graph.degree() if degree <= 1]
    
    print("Found endings: ", len(endings))
    # for every ending run BFS connection search
    i = 0
    while endings:
        # get point to find connection for
        start = endings.pop()
        # calculate search area
        r = edt_img[start] * multiplier
        search_area = int(4/3 * r * r * r * np.pi)
        # setup BFS
        points = SimpleQueue()
        points.put(start)
        visited = np.zeros_like(edt_img).astype(bool)
        # run BFS on a restricted area
        j = 0
        while not points.empty() and search_area:
            if j % 10 == 0:
                print(f"i: {i} j: {j} points: ", points.qsize())
            search_area -= 1
            # get point
            x, y, z = points.get()
            visited[x, y, z] = True
            
            # check if point is a node and is a valid connection
            if graph.has_node((x, y, z)) and not nx.has_path(graph, start, (x, y, z)):
                graph.add_edge(start, (x, y, z))
                # this is to prevent accidentally creating bifurcations
                if (x, y, z) in endings:
                    endings.pop(endings.index((x, y, z)))
                break

            # add point to search if it is in segmentation mask and it is not visited
            for dx in range(-1, 2):
                for dy in range(-1, 2):
                    for dz in range(-1, 2):
                        new_point = (x + dx, y + dy, z + dz)
                        if (
                            x + dx >= 0
                            and x + dx < edt_img.shape[0]
                            and y + dy >= 0
                            and y + dy < edt_img.shape[1]
                            and z + dz >= 0
                            and z + dz < edt_img.shape[2]
                            and new_point not in visited
                            and edt_img[x + dx, y + dy, z + dz] > 0
                        ):
                            visited[new_point] = True
                            points.put(new_point)
            j += 1
        i += 1

    return graph

In [ ]:
%%time
vessel_graph_cn = connect_endings_3d(vessel_graph, edt_img, 4)

Found endings:  98
i: 0 j: 0 points:  1
i: 0 j: 10 points:  252
i: 0 j: 20 points:  506
i: 0 j: 30 points:  741
i: 0 j: 40 points:  963
i: 0 j: 50 points:  1210
i: 0 j: 60 points:  1441
i: 0 j: 70 points:  1666
i: 0 j: 80 points:  1901
i: 0 j: 90 points:  2104
i: 0 j: 100 points:  2336
i: 0 j: 110 points:  2584
i: 0 j: 120 points:  2827
i: 0 j: 130 points:  3067
i: 0 j: 140 points:  3305
i: 0 j: 150 points:  3553
i: 0 j: 160 points:  3806
i: 0 j: 170 points:  4030
i: 0 j: 180 points:  4278
i: 0 j: 190 points:  4526
i: 0 j: 200 points:  4764
i: 0 j: 210 points:  5002
i: 0 j: 220 points:  5244
i: 0 j: 230 points:  5492
i: 0 j: 240 points:  5732
i: 0 j: 250 points:  5971
i: 0 j: 260 points:  6228
i: 0 j: 270 points:  6455
i: 0 j: 280 points:  6700
i: 0 j: 290 points:  6936
i: 0 j: 300 points:  7165
i: 0 j: 310 points:  7412
i: 0 j: 320 points:  7643
i: 0 j: 330 points:  7876
i: 0 j: 340 points:  8126
i: 0 j: 350 points:  8374
i: 0 j: 360 points:  8616
i: 0 j: 370 points:  8854
i: 0 j: 380

C:\Users\Alicja\AppData\Local\Temp\ipykernel_5248\3434991426.py:63: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  and new_point not in visited


i: 0 j: 710 points:  17112
i: 0 j: 720 points:  17369
i: 0 j: 730 points:  17618
i: 0 j: 740 points:  17806
i: 0 j: 750 points:  18018
i: 0 j: 760 points:  18241
i: 0 j: 770 points:  18467
i: 0 j: 780 points:  18674
i: 0 j: 790 points:  18893
i: 0 j: 800 points:  19115
i: 0 j: 810 points:  19330
i: 0 j: 820 points:  19555
i: 0 j: 830 points:  19765
i: 0 j: 840 points:  19995
i: 0 j: 850 points:  20219
i: 0 j: 860 points:  20439
i: 0 j: 870 points:  20682
i: 0 j: 880 points:  20897
i: 0 j: 890 points:  21127
i: 0 j: 900 points:  21375
i: 0 j: 910 points:  21582
i: 1 j: 0 points:  1
i: 1 j: 10 points:  256
i: 1 j: 20 points:  515
i: 1 j: 30 points:  756
i: 1 j: 40 points:  952
i: 1 j: 50 points:  1204
i: 1 j: 60 points:  1416
i: 1 j: 70 points:  1639
i: 1 j: 80 points:  1898
i: 1 j: 90 points:  2085
i: 1 j: 100 points:  2329
i: 1 j: 110 points:  2578
i: 1 j: 120 points:  2760
i: 1 j: 130 points:  3010
i: 1 j: 140 points:  3229
i: 1 j: 150 points:  3439
i: 1 j: 160 points:  3694
i: 1 j: 1

i: 1 j: 3980 points:  91384
i: 1 j: 3990 points:  91611
i: 1 j: 4000 points:  91803
i: 1 j: 4010 points:  92021
i: 1 j: 4020 points:  92223
i: 1 j: 4030 points:  92408
i: 1 j: 4040 points:  92617
i: 1 j: 4050 points:  92807
i: 1 j: 4060 points:  93015
i: 1 j: 4070 points:  93223
i: 1 j: 4080 points:  93411
i: 1 j: 4090 points:  93581
i: 1 j: 4100 points:  93813
i: 1 j: 4110 points:  94072
i: 1 j: 4120 points:  94248
i: 1 j: 4130 points:  94503
i: 1 j: 4140 points:  94743
i: 1 j: 4150 points:  94940
i: 1 j: 4160 points:  95184
i: 1 j: 4170 points:  95385
i: 1 j: 4180 points:  95612
i: 1 j: 4190 points:  95868
i: 1 j: 4200 points:  96034
i: 1 j: 4210 points:  96280
i: 1 j: 4220 points:  96518
i: 1 j: 4230 points:  96714
i: 1 j: 4240 points:  96939
i: 1 j: 4250 points:  97116
i: 1 j: 4260 points:  97364
i: 1 j: 4270 points:  97605
i: 1 j: 4280 points:  97800
i: 1 j: 4290 points:  98033
i: 1 j: 4300 points:  98230
i: 1 j: 4310 points:  98445
i: 1 j: 4320 points:  98686
i: 1 j: 4330 points:

i: 1 j: 6950 points:  158659
i: 1 j: 6960 points:  158908
i: 1 j: 6970 points:  159141
i: 1 j: 6980 points:  159368
i: 1 j: 6990 points:  159626
i: 1 j: 7000 points:  159872
i: 1 j: 7010 points:  160072
i: 1 j: 7020 points:  160296
i: 1 j: 7030 points:  160543
i: 1 j: 7040 points:  160716
i: 1 j: 7050 points:  160959
i: 1 j: 7060 points:  161179
i: 1 j: 7070 points:  161354
i: 1 j: 7080 points:  161594
i: 1 j: 7090 points:  161799
i: 1 j: 7100 points:  162027
i: 1 j: 7110 points:  162280
i: 1 j: 7120 points:  162459
i: 1 j: 7130 points:  162718
i: 1 j: 7140 points:  162959
i: 1 j: 7150 points:  163151
i: 1 j: 7160 points:  163406
i: 1 j: 7170 points:  163621
i: 1 j: 7180 points:  163837
i: 1 j: 7190 points:  164084
i: 1 j: 7200 points:  164269
i: 1 j: 7210 points:  164513
i: 1 j: 7220 points:  164764
i: 1 j: 7230 points:  164939
i: 2 j: 0 points:  1
i: 2 j: 10 points:  242
i: 2 j: 20 points:  480
i: 2 j: 30 points:  693
i: 2 j: 40 points:  889
i: 2 j: 50 points:  1092
i: 2 j: 60 points

...

i: 17 j: 5900 points:  144502
i: 17 j: 5910 points:  144732
i: 17 j: 5920 points:  144982
i: 17 j: 5930 points:  145207
i: 17 j: 5940 points:  145436
i: 17 j: 5950 points:  145693
i: 17 j: 5960 points:  145951
i: 17 j: 5970 points:  146205
i: 17 j: 5980 points:  146462
i: 17 j: 5990 points:  146718
i: 17 j: 6000 points:  146977
i: 17 j: 6010 points:  147224
i: 17 j: 6020 points:  147449
i: 17 j: 6030 points:  147699
i: 17 j: 6040 points:  147953
i: 17 j: 6050 points:  148201
i: 17 j: 6060 points:  148452
i: 18 j: 0 points:  1
i: 18 j: 10 points:  237
i: 18 j: 20 points:  484
i: 18 j: 30 points:  673
i: 18 j: 40 points:  873
i: 18 j: 50 points:  1122
i: 18 j: 60 points:  1316
i: 18 j: 70 points:  1573
i: 18 j: 80 points:  1772
i: 18 j: 90 points:  2023
i: 18 j: 100 points:  2234
i: 18 j: 110 points:  2465
i: 18 j: 120 points:  2696
i: 18 j: 130 points:  2914
i: 18 j: 140 points:  3169
i: 18 j: 150 points:  3378
i: 18 j: 160 points:  3619
i: 18 j: 170 points:  3849
i: 18 j: 180 points:  

i: 19 j: 610 points:  14376
i: 19 j: 620 points:  14594
i: 19 j: 630 points:  14830
i: 19 j: 640 points:  15037
i: 19 j: 650 points:  15288
i: 19 j: 660 points:  15485
i: 19 j: 670 points:  15743
i: 19 j: 680 points:  15955
i: 19 j: 690 points:  16191
i: 19 j: 700 points:  16423
i: 19 j: 710 points:  16605
i: 19 j: 720 points:  16830
i: 19 j: 730 points:  17048
i: 19 j: 740 points:  17266
i: 19 j: 750 points:  17488
i: 19 j: 760 points:  17698
i: 19 j: 770 points:  17920
i: 19 j: 780 points:  18138
i: 19 j: 790 points:  18363
i: 19 j: 800 points:  18605
i: 19 j: 810 points:  18816
i: 19 j: 820 points:  19046
i: 19 j: 830 points:  19258
i: 19 j: 840 points:  19481
i: 19 j: 850 points:  19698
i: 19 j: 860 points:  19916
i: 19 j: 870 points:  20144
i: 19 j: 880 points:  20367
i: 19 j: 890 points:  20596
i: 19 j: 900 points:  20815
i: 19 j: 910 points:  21046
i: 19 j: 920 points:  21271
i: 19 j: 930 points:  21518
i: 19 j: 940 points:  21753
i: 19 j: 950 points:  21978
i: 19 j: 960 points:

i: 20 j: 710 points:  15962
i: 20 j: 720 points:  16143
i: 20 j: 730 points:  16341
i: 20 j: 740 points:  16552
i: 20 j: 750 points:  16739
i: 20 j: 760 points:  16957
i: 20 j: 770 points:  17129
i: 20 j: 780 points:  17334
i: 20 j: 790 points:  17542
i: 20 j: 800 points:  17754
i: 20 j: 810 points:  17955
i: 20 j: 820 points:  18168
i: 20 j: 830 points:  18350
i: 20 j: 840 points:  18563
i: 20 j: 850 points:  18790
i: 20 j: 860 points:  19034
i: 20 j: 870 points:  19235
i: 20 j: 880 points:  19465
i: 20 j: 890 points:  19692
i: 20 j: 900 points:  19881
i: 20 j: 910 points:  20121
i: 20 j: 920 points:  20329
i: 20 j: 930 points:  20549
i: 20 j: 940 points:  20751
i: 20 j: 950 points:  20971
i: 20 j: 960 points:  21158
i: 20 j: 970 points:  21379
i: 20 j: 980 points:  21601
i: 20 j: 990 points:  21847
i: 20 j: 1000 points:  22052
i: 20 j: 1010 points:  22294
i: 20 j: 1020 points:  22519
i: 20 j: 1030 points:  22734
i: 20 j: 1040 points:  22975
i: 20 j: 1050 points:  23209
i: 20 j: 1060 

i: 22 j: 730 points:  17835
i: 22 j: 740 points:  18080
i: 22 j: 750 points:  18315
i: 22 j: 760 points:  18492
i: 22 j: 770 points:  18709
i: 22 j: 780 points:  18915
i: 22 j: 790 points:  19148
i: 22 j: 800 points:  19336
i: 22 j: 810 points:  19587
i: 22 j: 820 points:  19771
i: 22 j: 830 points:  20013
i: 22 j: 840 points:  20210
i: 22 j: 850 points:  20434
i: 22 j: 860 points:  20649
i: 22 j: 870 points:  20858
i: 22 j: 880 points:  21091
i: 22 j: 890 points:  21279
i: 22 j: 900 points:  21530
i: 22 j: 910 points:  21709
i: 22 j: 920 points:  21952
i: 22 j: 930 points:  22153
i: 22 j: 940 points:  22404
i: 22 j: 950 points:  22616
i: 22 j: 960 points:  22840
i: 22 j: 970 points:  23094
i: 22 j: 980 points:  23282
i: 22 j: 990 points:  23533
i: 22 j: 1000 points:  23784
i: 22 j: 1010 points:  23975
i: 22 j: 1020 points:  24229
i: 22 j: 1030 points:  24453
i: 22 j: 1040 points:  24668
i: 22 j: 1050 points:  24925
i: 22 j: 1060 points:  25122
i: 22 j: 1070 points:  25364
i: 22 j: 108

i: 22 j: 4270 points:  100598
i: 22 j: 4280 points:  100849
i: 22 j: 4290 points:  101100
i: 22 j: 4300 points:  101288
i: 22 j: 4310 points:  101548
i: 22 j: 4320 points:  101772
i: 22 j: 4330 points:  101982
i: 22 j: 4340 points:  102236
i: 22 j: 4350 points:  102421
i: 22 j: 4360 points:  102650
i: 22 j: 4370 points:  102888
i: 22 j: 4380 points:  103053
i: 22 j: 4390 points:  103293
i: 22 j: 4400 points:  103532
i: 22 j: 4410 points:  103791
i: 22 j: 4420 points:  104047
i: 22 j: 4430 points:  104307
i: 22 j: 4440 points:  104567
i: 22 j: 4450 points:  104827
i: 22 j: 4460 points:  105087
i: 22 j: 4470 points:  105347
i: 22 j: 4480 points:  105607
i: 22 j: 4490 points:  105867
i: 22 j: 4500 points:  106126
i: 22 j: 4510 points:  106386
i: 22 j: 4520 points:  106646
i: 22 j: 4530 points:  106906
i: 22 j: 4540 points:  107166
i: 22 j: 4550 points:  107426
i: 22 j: 4560 points:  107686
i: 22 j: 4570 points:  107940
i: 22 j: 4580 points:  108189
i: 22 j: 4590 points:  108430
i: 22 j: 4

In [112]:
# print(nx.info(vessel_graph_cn))

print('Number of nodes', len(vessel_graph_cn.nodes))
print('Number of edges', len(vessel_graph_cn.edges))
print('Average degree', sum(dict(vessel_graph_cn.degree).values()) / len(vessel_graph_cn.nodes))

Number of nodes 146
Number of edges 142
Average degree 1.9452054794520548


In [113]:
model_with_graph = draw_graph_on_model(scaled_mask, vessel_graph_cn)
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [114]:
%%time
data_graph = parametrize_graph(vessel_graph_cn, edt_img)

CPU times: total: 15.6 ms
Wall time: 15.2 ms


In [115]:
%%time
data_graph_cl = clean_data_graph(data_graph)

CPU times: total: 0 ns
Wall time: 988 µs


In [116]:
model_with_graph = draw_graph_on_model(scaled_mask, data_graph_cl)
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [ ]:
nx.get_edge_attributes(data_graph, "average_vessel_diameter").items(),